# Natural Language Processing

# Retrieval-Augmented generation (RAG)

RAG is a technique for augmenting LLM knowledge with additional, often private or real-time, data.

LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model’s cutoff date, you need to augment the knowledge of the model with the specific information it needs.

<img src="../figures/RAG-process.png" >

Introducing `ChakyBot`, an innovative chatbot designed to assist Chaky (the instructor) and TA (Gun) in explaining the lesson of the NLP course to students. Leveraging LangChain technology, ChakyBot excels in retrieving information from documents, ensuring a seamless and efficient learning experience for students engaging with the NLP curriculum.

1. Prompt
2. Retrieval
3. Memory
4. Chain

In [1]:
# #langchain library
# !pip install langchain==0.0.350
# #LLM
# !pip install accelerate==0.25.0
# !pip install transformers==4.36.2
# !pip install bitsandbytes==0.41.2
# #Text Embedding
# !pip install sentence-transformers==2.2.2
# !pip install InstructorEmbedding==1.0.1
# #vectorstore
# !pip install pymupdf==1.23.8
# !pip install faiss-gpu==1.7.2
# !pip install faiss-cpu==1.7.4

In [1]:
import os
import torch
# Set GPU device
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# os.environ['http_proxy']  = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

## 1. Prompt

A set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.

In [2]:
from langchain import PromptTemplate

prompt_template = """
    I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). 
    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. 
    Whether it's about probabilistic models, language models, or any other related topic, 
    I'm here to help break down complex concepts into easy-to-understand explanations.
    Just let me know what you're wondering about, and I'll do my best to guide you through it!
    {context}
    Question: {question}
    Answer:
    """.strip()

PROMPT = PromptTemplate.from_template(
    template = prompt_template
)

PROMPT
#using str.format 
#The placeholder is defined using curly brackets: {} {}

PromptTemplate(input_variables=['context', 'question'], template="I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). \n    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. \n    Whether it's about probabilistic models, language models, or any other related topic, \n    I'm here to help break down complex concepts into easy-to-understand explanations.\n    Just let me know what you're wondering about, and I'll do my best to guide you through it!\n    {context}\n    Question: {question}\n    Answer:")

In [3]:
PROMPT.format(
    context = "Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can effectively generalize and thus perform tasks without explicit instructions.",
    question = "What is Machine Learning"
)

"I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). \n    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. \n    Whether it's about probabilistic models, language models, or any other related topic, \n    I'm here to help break down complex concepts into easy-to-understand explanations.\n    Just let me know what you're wondering about, and I'll do my best to guide you through it!\n    Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can effectively generalize and thus perform tasks without explicit instructions.\n    Question: What is Machine Learning\n    Answer:"

Note : [How to improve prompting (Zero-shot, Few-shot, Chain-of-Thought, etc.](https://github.com/chaklam-silpasuwanchai/Natural-Language-Processing/blob/main/Code/05%20-%20RAG/advance/cot-tot-prompting.ipynb)

## 2. Retrieval

1. `Document loaders` : Load documents from many different sources (HTML, PDF, code). 
2. `Document transformers` : One of the essential steps in document retrieval is breaking down a large document into smaller, relevant chunks to enhance the retrieval process.
3. `Text embedding models` : Embeddings capture the semantic meaning of the text, allowing you to quickly and efficiently find other pieces of text that are similar.
4. `Vector stores`: there has emerged a need for databases to support efficient storage and searching of these embeddings.
5. `Retrievers` : Once the data is in the database, you still need to retrieve it.

### 2.1 Document Loaders 
Use document loaders to load data from a source as Document's. A Document is a piece of text and associated metadata. For example, there are document loaders for loading a simple .txt file, for loading the text contents of any web page, or even for loading a transcript of a YouTube video.

[PDF Loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)

[Download Document](https://web.stanford.edu/~jurafsky/slp3/)

In [9]:
from langchain.document_loaders import PyMuPDFLoader, PyPDFLoader

# nlp_docs = '../docs/pdf/SpeechandLanguageProcessing_3rd_07jan2023.pdf'

# loader = PyMuPDFLoader(nlp_docs)
# documents = loader.load()

folder_path = './docs/ait_pdf/'
pdf_list = []
for filename in os.listdir(folder_path):
    if filename.endswith('.pdf'):
        pdf_list.append(folder_path + filename)

documents = []
pdf_loaders = [PyPDFLoader(pdf) for pdf in pdf_list]
for loader in pdf_loaders:
    documents.extend(loader.load())

In [13]:
documents

In [11]:
len(documents)

636

In [12]:
documents[1]

Document(page_content='Summary of Contents\nI\nFundamental Algorithms for NLP\n1\n1\nIntroduction. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n3\n2\nRegular Expressions, Text Normalization, Edit Distance. . . . . . . . .\n4\n3\nN-gram Language Models . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n31\n4\nNaive Bayes, Text Classiﬁcation, and Sentiment . . . . . . . . . . . . . . . . . 58\n5\nLogistic Regression . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 79\n6\nVector Semantics and Embeddings . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 103\n7\nNeural Networks and Neural Language Models . . . . . . . . . . . . . . . . . 134\n8\nSequence Labeling for Parts of Speech and Named Entities . . . . . . 160\n9\nRNNs and LSTMs . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 185\n10 Transformers and Pretrained Langua

### 2.2 Document Transformers

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 100
)

doc = text_splitter.split_documents(documents)

In [16]:
doc[1]

Document(page_content='Summary of Contents\nI\nFundamental Algorithms for NLP\n1\n1\nIntroduction. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n3\n2\nRegular Expressions, Text Normalization, Edit Distance. . . . . . . . .\n4\n3\nN-gram Language Models . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .\n31\n4\nNaive Bayes, Text Classiﬁcation, and Sentiment . . . . . . . . . . . . . . . . . 58\n5\nLogistic Regression . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 79\n6\nVector Semantics and Embeddings . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 103\n7', metadata={'source': '../docs/pdf/SpeechandLanguageProcessing_3rd_07jan2023.pdf', 'file_path': '../docs/pdf/SpeechandLanguageProcessing_3rd_07jan2023.pdf', 'page': 1, 'total_pages': 636, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref', 'producer

In [17]:
len(doc)

3421

### 2.3 Text Embedding Models
Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

*Note* Instructor Model : [Huggingface](gingface.co/hkunlp/instructor-base) | [Paper](https://arxiv.org/abs/2212.09741)

In [5]:
import torch
from langchain.embeddings import HuggingFaceInstructEmbeddings

model_name = 'hkunlp/instructor-base'

embedding_model = HuggingFaceInstructEmbeddings(
    model_name = model_name,
    model_kwargs = {"device" : device}
)

C:\Users\putoo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\InstructorEmbedding\instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


### 2.4 Vector Stores

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector store takes care of storing embedded data and performing vector search for you.

In [22]:
#locate vectorstore
vector_path = '../A7/vector-store'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('create path done')

In [23]:
#save vector locally
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(
    documents = doc,
    embedding = embedding_model
)

db_file_name = 'nlp_stanford'

vectordb.save_local(
    folder_path = os.path.join(vector_path, db_file_name),
    index_name = 'nlp' #default index
)

### 2.5 retrievers
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

In [6]:
#calling vector from local
vector_path = '../A7/vector-store'
db_file_name = 'nlp_stanford'

from langchain.vectorstores import FAISS

vectordb = FAISS.load_local(
    folder_path = os.path.join(vector_path, db_file_name),
    embeddings = embedding_model,
    index_name = 'nlp' #default index
)   

In [7]:
#ready to use
retriever = vectordb.as_retriever()

In [8]:
# retriever.get_relevant_documents("What is Dependency Parsing")

retriever.get_relevant_documents("What is AIT")

[Document(page_content='68\nAIT ANNUAL REPORT 2011\nThe Asian Institute of Technology (AIT) has pro-\nvided graduate level education as per the devel-\nopment needs of countries in the Asian region for \nmore than 50 years. Today, with the emergence \nof some economies in Asia and the creation of in-\nstitutions possessing strong research infrastruc-\nture, AIT intends to create a niche for itself by \nbuilding rapidly on its existing strengths. \nThe research strategy developed in 2011 was the \nfirst of its kind in the history of AIT and has been \ndeveloped with this in mind. During the year, \nAIT Research Strategy 2012-2016 was created \nwith the understanding that research in today’s', metadata={'source': '../A7/docs/ait_pdf/AIT.AnnualReport.2011.pdf', 'file_path': '../A7/docs/ait_pdf/AIT.AnnualReport.2011.pdf', 'page': 69, 'total_pages': 152, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign CS5.5 (7.5)', 'producer': 'Adobe

In [9]:
# retriever.get_relevant_documents("What is Transformers")

retriever.get_relevant_documents("What is student exchange")

[Document(page_content='Factsheet \nSTUDENT \nEXCHANGE \nP R O G R A M M E \nSports & Fitness Centre \nHealth Care \nBank \nPost Ofﬁce \nComputing Facilities \nOther Facilities \nStudent Associations \nExchange services', metadata={'source': '../A7/docs/ait_pdf/IIMA-Factsheet-2024-25.pdf', 'file_path': '../A7/docs/ait_pdf/IIMA-Factsheet-2024-25.pdf', 'page': 3, 'total_pages': 4, 'format': 'PDF 1.7', 'title': 'Untitled-1.cdr', 'author': 'Designer IIMA', 'subject': '', 'keywords': '', 'creator': 'CorelDRAW 2021', 'producer': 'Corel PDF Engine Version 23.5.0.506', 'creationDate': "D:20240118172918+05'30'", 'modDate': "D:20240118172918+05'30'", 'trapped': ''}),
 Document(page_content='EXCHANGE \nPROGRAMS\nFull-time (22 months) Master’s degree \nstudents have the opportunity to take \npart in an exchange or with one of over \n50 of our partner universities around the \nworld. The aim of the exchange programs \nis to immerse students in another culture \nfor one or two semesters, and to expo

## 3. Memory

One of the core utility classes underpinning most (if not all) memory modules is the ChatMessageHistory class. This is a super lightweight wrapper that provides convenience methods for saving HumanMessages, AIMessages, and then fetching them all.

You may want to use this class directly if you are managing memory outside of a chain.


In [10]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history

ChatMessageHistory(messages=[])

In [11]:
history.add_user_message('hi')
history.add_ai_message('Whats up?')
history.add_user_message('How are you')
history.add_ai_message('I\'m quite good. How about you?')

In [38]:
history

ChatMessageHistory(messages=[HumanMessage(content='hi'), AIMessage(content='Whats up?'), HumanMessage(content='How are you'), AIMessage(content="I'm quite good. How about you?")])

### 3.1 Memory types

There are many different types of memory. Each has their own parameters, their own return types, and is useful in different scenarios. 
- Converstaion Buffer
- Converstaion Buffer Window

What variables get returned from memory

Before going into the chain, various variables are read from memory. These have specific names which need to align with the variables the chain expects. You can see what these variables are by calling memory.load_memory_variables({}). Note that the empty dictionary that we pass in is just a placeholder for real variables. If the memory type you are using is dependent upon the input variables, you may need to pass some in.

In this case, you can see that load_memory_variables returns a single key, history. This means that your chain (and likely your prompt) should expect an input named history. You can usually control this variable through parameters on the memory class. For example, if you want the memory variables to be returned in the key chat_history you can do:

#### Converstaion Buffer
This memory allows for storing messages and then extracts the messages in a variable.

In [12]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: hi\nAI: What's up?\nHuman: How are you?\nAI: I'm quite good. How about you?"}

In [13]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages = True)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'),
  AIMessage(content="What's up?"),
  HumanMessage(content='How are you?'),
  AIMessage(content="I'm quite good. How about you?")]}

#### Conversation Buffer Window
- it keeps a list of the interactions of the conversation over time. 
- it only uses the last K interactions. 
- it can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [14]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: How are you?\nAI: I'm quite good. How about you?"}

## 4. Chain

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

An `LLMChain` is a simple chain that adds some functionality around language models.
- it consists of a `PromptTemplate` and a `LM` (either an LLM or chat model).
- it formats the prompt template using the input key values provided (and also memory key values, if available), 
- it passes the formatted string to LLM and returns the LLM output.

Note : [Download Fastchat Model Here](https://huggingface.co/lmsys/fastchat-t5-3b-v1.0)

In [25]:
# %cd ./models
# !git clone https://huggingface.co/lmsys/fastchat-t5-3b-v1.0

%cd ./model_gpt2
!git clone https://huggingface.co/anas-awadalla/gpt2-span-head-few-shot-k-16-finetuned-squad-seed-0

In [61]:
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM, AutoModelForCausalLM, GPT2Tokenizer, GPT2Model
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
import torch

# model_id = '../A7/models/fastchat-t5-3b-v1.0/'

# model_id = 'D:/AIT-2023/NLP/A7/model_gpt2/gpt2-span-head-few-shot-k-16-finetuned-squad-seed-0'

model_id = '../A7/model_gpt2/gpt2-span-head-few-shot-k-16-finetuned-squad-seed-0'

tokenizer = AutoTokenizer.from_pretrained(model_id)

# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer.pad_token = tokenizer.eos_token

# bitsandbyte_config = BitsAndBytesConfig(
#     load_in_4bit = True,
#     bnb_4bit_quant_type = "nf4",
#     bnb_4bit_compute_dtype = torch.float16,
#     bnb_4bit_use_double_quant = True
# )

# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     # quantization_config=bitsandbyte_config,
#     device_map='cuda:0',
#     load_in_8bit=True
# )

model = AutoModelForCausalLM.from_pretrained(model_id)

# model = GPT2Model.from_pretrained('gpt2')

# model = GPT2Model.from_pretrained(
#     'gpt2',
#     # quantization_config = bitsandbyte_config, #caution Nvidia
#     quantization_config = None, #caution Nvidia
#     # device_map = 'auto',
#     load_in_8bit = True
# )

pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens = 50,
    model_kwargs = {
        "temperature" : 0,
        "repetition_penalty": 1.5
    }
)

llm = HuggingFacePipeline(pipeline = pipe)

### [Class ConversationalRetrievalChain](https://api.python.langchain.com/en/latest/_modules/langchain/chains/conversational_retrieval/base.html#ConversationalRetrievalChain)

- `retriever` : Retriever to use to fetch documents.

- `combine_docs_chain` : The chain used to combine any retrieved documents.

- `question_generator`: The chain used to generate a new question for the sake of retrieval. This chain will take in the current question (with variable question) and any chat history (with variable chat_history) and will produce a new standalone question to be used later on.

- `return_source_documents` : Return the retrieved source documents as part of the final result.

- `get_chat_history` : An optional function to get a string of the chat history. If None is provided, will use a default.

- `return_generated_question` : Return the generated question as part of the final result.

- `response_if_no_docs_found` : If specified, the chain will return a fixed response if no docs are found for the question.


`question_generator`

In [62]:
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain

In [63]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [64]:
question_generator = LLMChain(
    llm = llm,
    prompt = CONDENSE_QUESTION_PROMPT,
    verbose = True
)

In [65]:
query = 'Comparing both of them'
chat_history = "Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:"

question_generator({'chat_history' : chat_history, "question" : query})

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human:What is Machine Learning
AI:
Human:What is Deep Learning
AI:
Follow Up Input: Comparing both of them
Standalone question:

> Finished chain.


{'chat_history': 'Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:',
 'question': 'Comparing both of them',
 'text': ' Is Machine Learning An Important Question or Is It Just Some Random Thing About Which Computer Today Makes Us Human?Answer:Maybe, yeah.Machine learning is almost entirely based on direct comparisons of variables and tasks we use to solve real-world problems. For'}

`combine_docs_chain`

In [66]:
doc_chain = load_qa_chain(
    llm = llm,
    chain_type = 'stuff',
    prompt = PROMPT,
    verbose = True
)
doc_chain

StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). \n    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. \n    Whether it's about probabilistic models, language models, or any other related topic, \n    I'm here to help break down complex concepts into easy-to-understand explanations.\n    Just let me know what you're wondering about, and I'll do my best to guide you through it!\n    {context}\n    Question: {question}\n    Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x000001E4ACEF0450>)), document_variable_name='context')

In [67]:
# query = "What is Transformers?"

query = "What is AIT?"
input_document = retriever.get_relevant_documents(query)

doc_chain({'input_documents':input_document, 'question':query})

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). 
    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. 
    Whether it's about probabilistic models, language models, or any other related topic, 
    I'm here to help break down complex concepts into easy-to-understand explanations.
    Just let me know what you're wondering about, and I'll do my best to guide you through it!
    68
AIT ANNUAL REPORT 2011
The Asian Institute of Technology (AIT) has pro-
vided graduate level education as per the devel-
opment needs of countries in the Asian region for 
more than 50 years. Today, with the emergence 
of some economies in Asia and the creation of in-
stitutions possessing strong research infrastruc-
ture, AIT intends to create a 

{'input_documents': [Document(page_content='68\nAIT ANNUAL REPORT 2011\nThe Asian Institute of Technology (AIT) has pro-\nvided graduate level education as per the devel-\nopment needs of countries in the Asian region for \nmore than 50 years. Today, with the emergence \nof some economies in Asia and the creation of in-\nstitutions possessing strong research infrastruc-\nture, AIT intends to create a niche for itself by \nbuilding rapidly on its existing strengths. \nThe research strategy developed in 2011 was the \nfirst of its kind in the history of AIT and has been \ndeveloped with this in mind. During the year, \nAIT Research Strategy 2012-2016 was created \nwith the understanding that research in today’s', metadata={'source': '../A7/docs/ait_pdf/AIT.AnnualReport.2011.pdf', 'file_path': '../A7/docs/ait_pdf/AIT.AnnualReport.2011.pdf', 'page': 69, 'total_pages': 152, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign CS5.5 (7.5)'

In [68]:
memory = ConversationBufferWindowMemory(
    k=3, 
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True,
    get_chat_history=lambda h : h
)
chain

ConversationalRetrievalChain(memory=ConversationBufferWindowMemory(output_key='answer', return_messages=True, memory_key='chat_history', k=3), verbose=True, combine_docs_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). \n    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. \n    Whether it's about probabilistic models, language models, or any other related topic, \n    I'm here to help break down complex concepts into easy-to-understand explanations.\n    Just let me know what you're wondering about, and I'll do my best to guide you through it!\n    {context}\n    Question: {question}\n    Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerati

## 5. Chatbot

In [69]:
# prompt_question = "Who are you by the way?"

prompt_question = "What is student exchange?"
answer = chain({"question":prompt_question})
answer

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). 
    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. 
    Whether it's about probabilistic models, language models, or any other related topic, 
    I'm here to help break down complex concepts into easy-to-understand explanations.
    Just let me know what you're wondering about, and I'll do my best to guide you through it!
    Factsheet 
STUDENT 
EXCHANGE 
P R O G R A M M E 
Sports & Fitness Centre 
Health Care 
Bank 
Post Ofﬁce 
Computing Facilities 
Other Facilities 
Student Associations 
Exchange services

EXCHANGE 
PROGRAMS
Full-time (22 months) Master’s degree 
students have the opportunity to take 
part in an exc

{'question': 'What is student exchange?',
 'chat_history': [],
 'answer': ' Students at the University of \u200b\nThailand can take part in  exchange of students, faculty, and staff \nin a joint project.\nUniversity of \u200b \u200b \u200bThailand invites you to explore\nuniversities around the world and collaborate',
 'source_documents': [Document(page_content='Factsheet \nSTUDENT \nEXCHANGE \nP R O G R A M M E \nSports & Fitness Centre \nHealth Care \nBank \nPost Ofﬁce \nComputing Facilities \nOther Facilities \nStudent Associations \nExchange services', metadata={'source': '../A7/docs/ait_pdf/IIMA-Factsheet-2024-25.pdf', 'file_path': '../A7/docs/ait_pdf/IIMA-Factsheet-2024-25.pdf', 'page': 3, 'total_pages': 4, 'format': 'PDF 1.7', 'title': 'Untitled-1.cdr', 'author': 'Designer IIMA', 'subject': '', 'keywords': '', 'creator': 'CorelDRAW 2021', 'producer': 'Corel PDF Engine Version 23.5.0.506', 'creationDate': "D:20240118172918+05'30'", 'modDate': "D:20240118172918+05'30'", 'trapped':

In [70]:
# prompt_question = "What is the Transformers?"

prompt_question = "What is student exchange?"
answer = chain({"question":prompt_question})
answer

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='What is student exchange?'), AIMessage(content=' Students at the University of \u200b\nThailand can take part in  exchange of students, faculty, and staff \nin a joint project.\nUniversity of \u200b \u200b \u200bThailand invites you to explore\nuniversities around the world and collaborate')]
Follow Up Input: What is student exchange?
Standalone question:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). 
    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. 
    Whether it's about probabilistic models, language models, or any other related topic, 
    I'm here to help break down complex concepts into easy-to-understand explanations.
    Just let me know what you're wondering about, and I'll do my best to guide you through it!
    Factsheet 
STUDENT 
EXCHANGE 
P R O G R A M M E 
Sports & Fitness Centre 
Health Care 
Bank 
Post Ofﬁce 
Computing Facilities 
Other Facilities 
Student Associations 
Exchange services

2.4.2 Data and Information Collection 
2.5 
Quality Assurance 
2.6 
Gender Sensitivity/Care
2.7 
Limitations of the Assessment 
Chapter 3: Socio Ec

{'question': 'What is student exchange?',
 'chat_history': [HumanMessage(content='What is student exchange?'),
  AIMessage(content=' Students at the University of \u200b\nThailand can take part in  exchange of students, faculty, and staff \nin a joint project.\nUniversity of \u200b \u200b \u200bThailand invites you to explore\nuniversities around the world and collaborate')],
 'answer': ' student exchange has always been an    effort by some schools to discourage\nthose in power from pursuing career options for which they didn\\t even consider pursuing them\naccording to conservative estimates. However, while some colleges may admit as many as\n',
 'source_documents': [Document(page_content='Factsheet \nSTUDENT \nEXCHANGE \nP R O G R A M M E \nSports & Fitness Centre \nHealth Care \nBank \nPost Ofﬁce \nComputing Facilities \nOther Facilities \nStudent Associations \nExchange services', metadata={'source': '../A7/docs/ait_pdf/IIMA-Factsheet-2024-25.pdf', 'file_path': '../A7/docs/ait_pdf

In [71]:
prompt_question = "Where is AIT?"
answer = chain({"question":prompt_question})
answer

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='What is student exchange?'), AIMessage(content=' Students at the University of \u200b\nThailand can take part in  exchange of students, faculty, and staff \nin a joint project.\nUniversity of \u200b \u200b \u200bThailand invites you to explore\nuniversities around the world and collaborate'), HumanMessage(content='What is student exchange?'), AIMessage(content=' student exchange has always been an    effort by some schools to discourage\nthose in power from pursuing career options for which they didn\\t even consider pursuing them\naccording to conservative estimates. However, while some colleges may admit as many as\n')]
Follow Up Input: Where is AIT?
Standalone question:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). 
    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. 
    Whether it's about probabilistic models, language models, or any other related topic, 
    I'm here to help break down complex concepts into easy-to-understand explanations.
    Just let me know what you're wondering about, and I'll do my best to guide you through it!
    simulation of human intelligence.  AI has two 
primary "levels".  The first level, "narrow AI", 
concerns perception, statistical inference, and 
actuation, drawing on data science, sensors, and 
robotics.  The second level, sometimes called 
"a
"artificial general intelligence (AGI), is concerned 
with more complex or flexible rea

{'question': 'Where is AIT?',
 'chat_history': [HumanMessage(content='What is student exchange?'),
  AIMessage(content=' Students at the University of \u200b\nThailand can take part in  exchange of students, faculty, and staff \nin a joint project.\nUniversity of \u200b \u200b \u200bThailand invites you to explore\nuniversities around the world and collaborate'),
  HumanMessage(content='What is student exchange?'),
  AIMessage(content=' student exchange has always been an    effort by some schools to discourage\nthose in power from pursuing career options for which they didn\\t even consider pursuing them\naccording to conservative estimates. However, while some colleges may admit as many as\n')],
 'answer': "asked, and after some deliberation,36 39 ORDERING THAT THE COMMITTEE SAMPLES (2)A description of The National Science Foundation As of 24 December 2013, the followingNASS, which is the federal government's official scientific",
 'source_documents': [Document(page_content='simulati